# Gensim을 사용한 Word2Vec 모델 학습
* 데이터는 NSMC를 사용

In [1]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
from gensim.models import Word2Vec

In [2]:
def read_review(filename):
    f = open(filename,'r',encoding = 'utf-8')
    data = []
    for line in f.readlines():
        data.append(line.split('\t'))
    data = data[1:]
    f.close()
    return data

In [3]:
data = read_review('ratings.txt')

In [4]:
data[0]

['8112052', '어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '1\n']

In [5]:
from konlpy.tag import Komoran
komoran = Komoran()
docs = []
for sent in data:
    docs.append(komoran.morphs(sent[1]))
print(docs[0])

['어리', 'ㄹ', '때', '보고', '지금', '다시', '보', '아도', '재밌어요ㅋㅋ']


In [7]:
docs[1]

['디자인',
 '을',
 '배우',
 '는',
 '학생',
 '으로',
 ',',
 '외국',
 '디자이너',
 '와',
 '그',
 '들',
 '이',
 '일구',
 'ㄴ',
 '전통',
 '을',
 '통하',
 '아',
 '발전',
 '하',
 '아',
 '가',
 '는',
 '문화',
 '산업',
 '이',
 '부럽',
 '었',
 '는데',
 '.',
 '사실',
 '우리나라',
 '에서',
 '도',
 '그',
 '어렵',
 'ㄴ',
 '시절',
 '에',
 '끝',
 '까지',
 '열정',
 '을',
 '지키',
 'ㄴ',
 '노라',
 '노',
 '같',
 '은',
 '전통',
 '이',
 '있',
 '어',
 '저',
 '와',
 '같',
 '은',
 '사람',
 '들',
 '이',
 '꿈',
 '을',
 '꾸',
 '고',
 '이루',
 '어',
 '나가',
 'ㄹ',
 '수',
 '있',
 '다는',
 '것',
 '에',
 '감사',
 '하',
 'ㅂ니다',
 '.']

## from gensim.models import Word2Vec 의 파라미터
### Word2Vec의 mode에는 skip-gram과 CBOW가 있음
* sentences: Word2Vec모델 학습에 필요한 문장 데이터
* size: 임베딩된 벡터의 차원
* window: 주변 단어 윈도우의 크기
* hs: 0 -> negative sampling / 1 -> hierarchical softmax
    * Word2Vec은 다른 기법들을 사용해서 계산 복잡도를 줄일 수 있다.
* min_count: 단어 최소 빈도수 제한( 한번 등장하는 것에 대해서도 word embedding vector를 만드는 것을 제한한다)
* sg: 0-> CBOW , 1-> skip-gram

In [8]:
# sentences로 들어가는 것은 2차원 코퍼스 리스트이다.
import time
start = time.time()
model = Word2Vec(sentences = docs,
                vector_size = 200,
                window = 4, # 앞 뒤 단어 2개씩 사용 context window
                hs = 1, 
                min_count = 5,
                sg = 1)# skipgram을 사용
print("Done..",time.time()-start)

Done.. 41.25443696975708


In [9]:
model.save('nsmc.model') # model을 저장

In [10]:
model = Word2Vec.load('nsmc.model')

* wv 속성을 이용해서, 임베딩 벡터값을 가져오게 된다.

In [11]:
model.wv

In [12]:
print('영화에 대한 임베딩 벡터:',model.wv['영화'])

영화에 대한 임베딩 벡터: [-0.13448098  0.03660773 -0.12252025 -0.12427539  0.20397884 -0.32054397
  0.08807015  0.13284434 -0.14763173  0.15459657  0.00246582  0.17123257
  0.05329295  0.12461527 -0.3016517  -0.02805172  0.18334487  0.2953543
  0.05367627 -0.4131216   0.04810801 -0.0631011   0.04434726  0.09189259
  0.02201813  0.05853794  0.06420454 -0.12644134 -0.1782414   0.19221807
  0.12618273  0.2920489   0.1263492   0.3778864   0.1646944   0.08852613
  0.16206841  0.02641144 -0.29579547 -0.24754612  0.10527302  0.01544663
 -0.1864093   0.24227206  0.11042933  0.22289275 -0.16097951 -0.1939594
  0.21840546  0.05563221  0.00877432  0.01499947 -0.01747849  0.28902614
  0.09176775 -0.22039242 -0.08413702 -0.155536   -0.38297236 -0.19620436
 -0.22990921  0.06618039  0.21296394 -0.02125058 -0.01414576 -0.11035353
  0.15443417 -0.11205962 -0.25821579  0.01597224  0.27115083  0.24384104
  0.02691566  0.08804599 -0.12412421 -0.1414661   0.03008161 -0.02067515
 -0.00357299 -0.03360223 -0.11781909  

In [13]:
model.wv.similarity('영화','배우')

0.355527

In [14]:
model.wv.most_similar('영화',topn=10)

[('드라마', 0.6260667443275452),
 ('쓰레기', 0.5983583331108093),
 ('공포물', 0.5768200159072876),
 ('애니', 0.5763930678367615),
 ('B급 영화', 0.5574089288711548),
 ('쓰렉', 0.5417054891586304),
 ('애니메이션', 0.5416151881217957),
 ('핵폐기물', 0.5391795039176941),
 ('무협지', 0.5373629927635193),
 ('쓰.레.기', 0.535732090473175)]

# Result
* 유사한 단어를 잘 찾는 경우도 있지만 아닌 경우도 있다.

**이는 보통 문장 말뭉치(corpus)가 부족하거나, 퀄리티 높은 문장 부족등의 문제가 있다.**<br>**&rightarrow; 따라서, 충분히 많은 퀄리티 좋은 문장corpus가 있다면 좋은 임베딩 결과를 낼 수 있다.**